In [1]:
import cogsworth
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# this all just makes plots look nice
%config InlineBackend.figure_format = 'retina'

plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
fs = 24

# update various fontsizes to match
params = {'figure.figsize': (12, 8),
          'legend.fontsize': fs,
          'axes.labelsize': fs,
          'xtick.labelsize': 0.9 * fs,
          'ytick.labelsize': 0.9 * fs,
          'axes.linewidth': 1.1,
          'xtick.major.size': 7,
          'xtick.minor.size': 4,
          'ytick.major.size': 7,
          'ytick.minor.size': 4}
plt.rcParams.update(params)
pd.options.display.max_columns = 999

In [3]:
p = cogsworth.pop.Population(n_binaries=100,
                             final_kstar1=[13, 14], final_kstar2=[13, 14],
                             BSE_settings={"sigma": 30})
p.sample_initial_binaries()
p.perform_stellar_evolution()

First let's see what fraction of the population were disrupted when drawing kicks from a Maxwellian centred on $30 \, {\rm km / s}$

In [4]:
# count the number of binaries that got kicks while they hadn't already merged
n_kicked = len(p.bpp[((p.bpp["evol_type"] == 15) | (p.bpp["evol_type"] == 16)) & (p.bpp["sep"] != 0.0)]["bin_num"].unique())
f'{p.disrupted.sum() / n_kicked:1.2f}'

'0.40'

Now let's change the Maxwellian to be centred on $1000 \, {\rm km / s}$ and do the same for electron-capture supernovae.

In [5]:
p.BSE_settings["sigma"] = 1000
p.BSE_settings["sigmadiv"] = -1000
p.perform_stellar_evolution()

In [6]:
# count the number of binaries that got kicks while they hadn't already merged
n_kicked = len(p.bpp[((p.bpp["evol_type"] == 15) | (p.bpp["evol_type"] == 16)) & (p.bpp["sep"] != 0.0)]["bin_num"].unique())
f'{p.disrupted.sum() / n_kicked:1.2f}'

'0.93'

As you'd expect, the fraction is now much higher (and any that aren't disrupting are likely getting fallback adjusted kicks, got lucky directions, or had extremely high orbital velocities).

In [18]:
# create a population of 100 binaries
p = cogsworth.pop.Population(n_binaries=1000)
p.sample_initial_binaries()

# for a series of different alphas
for alpha in [10, 2, 1, 0.5, 0.1]:
    # change the setting and perform evolution
    p.BSE_settings["alpha1"] = alpha
    p.perform_stellar_evolution()
    
    # find all binaries that start a CE and how many end as a mergers
    ce_nums = p.bpp[p.bpp["evol_type"] == 7]["bin_num"].unique()
    n_merged = (p.final_bpp.loc[ce_nums]["sep"] == 0.0).sum()
    
    # get separations after the first CE for each binary
    seps = p.bpp[p.bpp["evol_type"] == 8].drop_duplicates(subset="bin_num", keep="first")["sep"].values
    
    print(f"Run for alpha = {alpha}")
    print(f"  {len(ce_nums)} binaries experience at least one common-envelope event")
    print(f"  {n_merged / len(ce_nums) * 100:1.1f}% of these resulted in a stellar merger")
    print(f"  For surviving systems, mean separation immediately after the CE is {np.mean(seps):1.2f} Rsun\n")

Run for alpha = 10
  85 binaries experience at least one common-envelope event
  25.9% of these resulted in a stellar merger
  For surviving systems, mean separation immediately after the CE is 32.97 Rsun

Run for alpha = 2
  85 binaries experience at least one common-envelope event
  64.7% of these resulted in a stellar merger
  For surviving systems, mean separation immediately after the CE is 9.07 Rsun

Run for alpha = 1
  85 binaries experience at least one common-envelope event
  76.5% of these resulted in a stellar merger
  For surviving systems, mean separation immediately after the CE is 4.58 Rsun

Run for alpha = 0.5
  85 binaries experience at least one common-envelope event
  83.5% of these resulted in a stellar merger
  For surviving systems, mean separation immediately after the CE is 1.96 Rsun

Run for alpha = 0.1
  85 binaries experience at least one common-envelope event
  94.1% of these resulted in a stellar merger
  For surviving systems, mean separation immediately a